# 🦙 Usar Modelo Fine-tuneado con Ollama

## Meta Day Uruguay 2025 - Conexión Módulo 4 → Módulo 5

Este notebook muestra cómo usar el modelo que fine-tuneaste en el notebook anterior con **Ollama** para crear un sistema de chat local.

### Prerrequisitos:
1. Haber completado el fine-tuning con Unsloth
2. Tener los archivos GGUF exportados
3. Ollama instalado localmente

### Lo que aprenderás:
- Importar tu modelo fine-tuneado a Ollama
- Crear un Modelfile personalizado
- Configurar parámetros de inferencia
- Probar el modelo en un chat interactivo

## 🔧 Configuración Inicial

### Verificar instalación de Ollama

In [ ]:
import subprocess
import os
import json
from pathlib import Path

def run_command(command):
    """Ejecuta un comando y retorna el resultado"""
    try:
        result = subprocess.run(command, shell=True, capture_output=True, text=True)
        return result.stdout.strip(), result.stderr.strip(), result.returncode
    except Exception as e:
        return "", str(e), 1

# Verificar si Ollama está instalado
stdout, stderr, code = run_command("ollama --version")
if code == 0:
    print(f"✅ Ollama instalado: {stdout}")
else:
    print("❌ Ollama no está instalado")
    print("📥 Descarga desde: https://ollama.ai")
    print("💡 O instala con: curl -fsSL https://ollama.ai/install.sh | sh")

## 📝 Crear Modelfile Personalizado

Un Modelfile define cómo Ollama debe cargar y configurar tu modelo:

In [ ]:
# Configuración del modelo
model_name = "llama3.1-metaday-finetune"
gguf_path = "./gguf_model/model-q4_k_m.gguf"  # Ajusta la ruta según tu exportación

# Crear Modelfile
modelfile_content = f"""# Modelo Llama 3.1 Fine-tuneado - Meta Day Uruguay 2025
FROM {gguf_path}

# Template de chat (ChatML)
TEMPLATE """<|im_start|>system
{{ .System }}<|im_end|>
<|im_start|>user
{{ .Prompt }}<|im_end|>
<|im_start|>assistant
"""

# Parámetros de inferencia optimizados
PARAMETER temperature 0.7
PARAMETER top_p 0.9
PARAMETER top_k 40
PARAMETER repeat_penalty 1.1
PARAMETER num_ctx 2048

# Mensaje del sistema
SYSTEM """Eres un asistente de IA útil y conocedor, entrenado durante el Meta Day Uruguay 2025. 
Respondes de manera clara, precisa y educativa. Siempre intentas ser útil y proporcionar 
información valiosa al usuario."""
"""

# Guardar Modelfile
with open("Modelfile", "w", encoding="utf-8") as f:
    f.write(modelfile_content)

print("✅ Modelfile creado")
print("📄 Contenido:")
print("=" * 50)
print(modelfile_content)
print("=" * 50)

## 📦 Importar Modelo a Ollama

In [ ]:
# Verificar que el archivo GGUF existe
if os.path.exists(gguf_path):
    print(f"✅ Archivo GGUF encontrado: {gguf_path}")
    
    # Crear el modelo en Ollama
    print(f"📦 Importando modelo '{model_name}' a Ollama...")
    stdout, stderr, code = run_command(f"ollama create {model_name} -f Modelfile")
    
    if code == 0:
        print("🎉 ¡Modelo importado exitosamente!")
        print(f"📝 Salida: {stdout}")
    else:
        print(f"❌ Error al importar: {stderr}")
else:
    print(f"❌ Archivo GGUF no encontrado: {gguf_path}")
    print("💡 Asegúrate de haber ejecutado la exportación GGUF en el notebook anterior")
    print("💡 O ajusta la variable 'gguf_path' con la ruta correcta")

## 📋 Verificar Modelos Disponibles

In [ ]:
# Listar modelos en Ollama
print("📋 Modelos disponibles en Ollama:")
stdout, stderr, code = run_command("ollama list")

if code == 0:
    print(stdout)
    
    # Verificar si nuestro modelo está en la lista
    if model_name in stdout:
        print(f"\n✅ Tu modelo '{model_name}' está listo para usar")
    else:
        print(f"\n⚠️ Modelo '{model_name}' no encontrado en la lista")
else:
    print(f"❌ Error al listar modelos: {stderr}")

## 🧪 Probar el Modelo

In [ ]:
def chat_with_model(prompt, model=model_name):
    """Función para chatear con el modelo"""
    print(f"🤖 Pregunta: {prompt}")
    print(f"💭 Respuesta de {model}:")
    print("-" * 50)
    
    # Ejecutar ollama run
    command = f'ollama run {model} "{prompt}"'
    stdout, stderr, code = run_command(command)
    
    if code == 0:
        print(stdout)
    else:
        print(f"❌ Error: {stderr}")
    
    print("=" * 50)

# Pruebas del modelo
test_prompts = [
    "¿Qué aprendiste durante tu fine-tuning en el Meta Day Uruguay 2025?",
    "Explica qué es LoRA en términos simples",
    "¿Cuáles son las ventajas de usar Unsloth para fine-tuning?",
    "¿Es 9.11 mayor que 9.9? Explica tu razonamiento."
]

print("🧪 Probando el modelo fine-tuneado...\n")

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n🔍 Prueba {i}/{len(test_prompts)}")
    chat_with_model(prompt)

print("\n✅ Todas las pruebas completadas")

## 💬 Chat Interactivo (Opcional)

Para un chat más interactivo, puedes usar la terminal:

In [ ]:
print("💬 Para iniciar un chat interactivo, ejecuta en tu terminal:")
print(f"   ollama run {model_name}")
print("\n🔧 Comandos útiles:")
print(f"   ollama show {model_name}           # Ver información del modelo")
print(f"   ollama rm {model_name}             # Eliminar el modelo")
print(f"   ollama pull llama3.1               # Descargar modelo base")
print(f"   ollama list                        # Listar todos los modelos")

print("\n🌐 También puedes usar la API REST de Ollama:")
print("   curl http://localhost:11434/api/generate -d '{")
print(f'     "model": "{model_name}",")
print('     "prompt": "Tu pregunta aquí"')
print("   }'")

## 🔌 Ejemplo de API REST

In [ ]:
import requests
import json

def chat_via_api(prompt, model=model_name):
    """Usar la API REST de Ollama"""
    url = "http://localhost:11434/api/generate"
    data = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }
    
    try:
        response = requests.post(url, json=data, timeout=30)
        if response.status_code == 200:
            result = response.json()
            return result.get("response", "Sin respuesta")
        else:
            return f"Error HTTP: {response.status_code}"
    except requests.exceptions.RequestException as e:
        return f"Error de conexión: {e}"

# Probar API (solo si Ollama está ejecutándose)
print("🔌 Probando API REST de Ollama...")
test_prompt = "Hola, ¿cómo estás?"
response = chat_via_api(test_prompt)
print(f"Pregunta: {test_prompt}")
print(f"Respuesta: {response}")

## 🎯 Resumen y Próximos Pasos

### ✅ Lo que hemos logrado:
- Importado tu modelo fine-tuneado a Ollama
- Configurado un Modelfile personalizado con ChatML template
- Probado el modelo con diferentes preguntas
- Aprendido a usar la API REST de Ollama

### 🚀 Conexión con Módulo 5 (RAG):
Ahora puedes usar este modelo fine-tuneado en el **Módulo 5** para:
- Crear un sistema RAG personalizado
- Combinar tu conocimiento fine-tuneado con documentos externos
- Construir un chatbot especializado para tu dominio

### 💡 Ideas para expandir:
1. **Integrar con LangChain**: Usar tu modelo en pipelines RAG
2. **Crear una interfaz web**: Usar Streamlit o Gradio
3. **Optimizar parámetros**: Ajustar temperature, top_p según tu uso
4. **Monitorear rendimiento**: Comparar con modelo base
5. **Escalar**: Probar con modelos más grandes (70B)

### 📚 Recursos útiles:
- [Ollama Documentation](https://github.com/ollama/ollama) - Documentación oficial
- [Modelfile Reference](https://github.com/ollama/ollama/blob/main/docs/modelfile.md) - Referencia completa
- [Ollama API](https://github.com/ollama/ollama/blob/main/docs/api.md) - Documentación de API

---
**Meta Day Uruguay 2025** - Puente Módulo 4 → Módulo 5 🇺🇾